In [35]:
import numpy as np

np.set_printoptions(linewidth=200)


data = """
##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""


def get_next_space(grid, robot, m):
    move = {
        "<": (0, -1),
        "^": (-1, 0),
        ">": (0, 1),
        "v": (1, 0),
    }

    sr, sc = np.array(robot) + np.array(move[m])
    item = grid[sr, sc]
    if item == "#":
        return (sr, sc), False
    elif item == ".":
        return (sr, sc), True
    elif item == "O":
        return get_next_space(grid, (sr, sc), m)


raw_grid, raw_moves = data.strip().split("\n\n")
grid = np.array([list(r) for r in raw_grid.split("\n")])
moves = raw_moves.replace("\n", "")

rr, rc = list(zip(*np.where(grid == "@")))[0]

for m in moves:
    coords, is_move = get_next_space(grid, (rr, rc), m)
    nr, nc = coords
    
    if m == ">" and is_move:
        move_group = grid[rr:nr + 1, rc:nc].copy()
        grid[rr:nr + 1, rc:nc] = "."
        grid[rr:nr + 1, rc + 1:nc + 1] = move_group
        rr, rc = rr, rc + 1
    elif m == "<" and is_move:
        move_group = grid[rr:nr + 1, nc + 1:rc + 1].copy()
        grid[rr:nr + 1, nc + 1:rc + 1] = "."
        grid[rr:nr + 1, nc:rc] = move_group
        rr, rc = rr, rc - 1
    elif m == "v" and is_move:
        move_group = grid[rr:nr, rc:nc + 1].copy()
        grid[rr:nr, rc:nc + 1] = "."
        grid[rr + 1:nr + 1, rc:nc + 1] = move_group
        rr, rc = rr + 1, rc
    elif m == "^" and is_move:
        move_group = grid[nr + 1:rr + 1, rc:nc + 1].copy()
        grid[nr + 1:rr + 1, rc:nc + 1] = "."
        grid[nr:rr, rc:nc + 1] = move_group
        rr, rc = rr - 1, rc


brs, bcs = np.where(grid == "O")

np.sum(brs * 100 + bcs)

10092

In [39]:
data = """
########
#..O...#
##..O..#
#...O..#
#.#.O..#
#...@#.#
#......#
########

<vv>^<v^>^"""

expand = {
'#': '##',
'O': '[]',
'.': '..',
'@': '@.',
}

raw_grid, raw_moves = data.strip().split("\n\n")
for orig, exp in expand.items():
    raw_grid = raw_grid.replace(orig, exp)

grid_ = np.array([list(r) for r in raw_grid.split("\n")])
moves = raw_moves.replace("\n", "")
rr, rc = list(zip(*np.where(grid == "@")))[0]

grid_

array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '#', '#', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '#', '#', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '@', '.', '#', '#', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [40]:
move = {
    "<": (0, -1),
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
}


def get_next_space(grid, robot, m, to_move):
    sr, sc = np.array(robot) + np.array(move[m])
    item = grid[sr, sc]
    to_move.append(robot)
    if item == "#":
        return (sr, sc), False, to_move
    elif item == ".":
        return (sr, sc), True, to_move
    elif m in "v^" and item == "[":
        left, is_left_move, to_move = get_next_space(grid, (sr, sc), m, to_move)
        right, is_right_move, to_move = get_next_space(grid, (sr, sc + 1), m, to_move)
        return left, is_left_move and is_right_move, to_move
    elif m in "v^" and item == "]":
        left, is_left_move, to_move = get_next_space(grid, (sr, sc - 1), m, to_move)
        right, is_right_move, to_move = get_next_space(grid, (sr, sc), m, to_move)
        return right, is_left_move and is_right_move, to_move
    elif (m in "<>" and item in "[]") or item == "O":
        return get_next_space(grid, (sr, sc), m, to_move)
        
grid = np.array([['.', '@', '.', '.'], ['.', '[', ']', '.'], ['[', ']', '[', ']'], ['#', '.', '.', '.'], ['.', '.', '.', '.']])
print(grid)
get_next_space(grid, (0, 1), "v", list())

[['.' '@' '.' '.']
 ['.' '[' ']' '.']
 ['[' ']' '[' ']']
 ['#' '.' '.' '.']
 ['.' '.' '.' '.']]


((3, 1), False, [(0, 1), (1, 1), (2, 0), (2, 1), (1, 2), (2, 2), (2, 3)])

In [41]:
# for m in moves:
m = "^"
grid = grid_

grid

array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '#', '#', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '#', '#', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '@', '.', '#', '#', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [42]:
coords, is_move, to_move = get_next_space(grid, (rr, rc), m, list())
orig = {(mr, mc): grid[mr, mc] for mr, mc in to_move}
nr, nc = move[m]

for mr, mc in to_move:
    grid[mr, mc] = "."

for mr, mc in to_move:
    grid[mr + nr, mc + nc] = orig[(mr, mc)]
    

grid

array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '[', ']', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '#', '#', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '[', ']', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '#', '#', '.', '.', '@', '.', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '.', '.', '#', '#'],
       ['#', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U1')